In [1]:
import os
os.getcwd()

'C:\\Users\\Admin\\Documents\\DataScience\\TECHCRUSH\\Resources\\pizza_sales'

# Pizza Place Sales — Analysis Notebook

**Goal:** Analyze one year of pizza shop sales from four CSV files (Orders, Order Details, Pizzas, Pizza Types).  
The goal is to clean & merge the provided data into a master DataFrame, perform EDA and answer the recommended questions (total revenue, quantity sold, orders, top pizzas, peak hours, monthly trends, underperforming pizza types)

In [2]:
#--------IMPORT LIBRARIES---------------------

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# ML / utilities (optional later)
from sklearn.preprocessing import StandardScaler

# Notebook display settings
pd.set_option('display.max_columns', None) #show all columns
pd.set_option('display.width', None) # unwrap texts
sns.set(style='whitegrid')


import warnings
warnings.filterwarnings("ignore")

In [3]:
#-----------READ CSV FILES---------------

orders = pd.read_csv("orders.csv")
print(f"------------Orders loaded successfully. Shape: {orders.shape}.----------------------")
print(orders.head())

order_details = pd.read_csv("order_details.csv")
print(f"------------Order_details loaded successfully. Shape: {order_details.shape}---------")
print(order_details.head())

pizzas = pd.read_csv("pizzas.csv")
print(f"------------Pizzas loaded successfully. Shape: {pizzas.shape}-----------------------")
print(pizzas.head())

# Error in reading the "pizza_types" document hence the encoding error argument
pizza_types = pd.read_csv("pizza_types.csv", encoding_errors="replace") 
print(f"------------Pizza types loaded successfully. Size: {pizza_types.shape}--------------")
print(pizza_types.head())

------------Orders loaded successfully. Shape: (21350, 3).----------------------
   order_id        date      time
0         1  2015-01-01  11:38:36
1         2  2015-01-01  11:57:40
2         3  2015-01-01  12:12:28
3         4  2015-01-01  12:16:31
4         5  2015-01-01  12:21:30
------------Order_details loaded successfully. Shape: (48620, 4)---------
   order_details_id  order_id       pizza_id  quantity
0                 1         1     hawaiian_m         1
1                 2         2  classic_dlx_m         1
2                 3         2  five_cheese_l         1
3                 4         2    ital_supr_l         1
4                 5         2     mexicana_m         1
------------Pizzas loaded successfully. Shape: (96, 4)-----------------------
     pizza_id pizza_type_id size  price
0   bbq_ckn_s       bbq_ckn    S  12.75
1   bbq_ckn_m       bbq_ckn    M  16.75
2   bbq_ckn_l       bbq_ckn    L  20.75
3  cali_ckn_s      cali_ckn    S  12.75
4  cali_ckn_m      cali_ckn    M 

In [4]:
#--------JOIN .CSV FILES---------

# Merge orders with order_details on 'order_id'
orders_order_details = pd.merge(order_details, orders, on='order_id', how='inner')
print("--------orders_orders_details merged successfully------")

# Merge orders_order_details with pizzas on 'pizza_id'
orders_pizzas = pd.merge(orders_order_details, pizzas, on='pizza_id', how='inner')
print("--------orders_pizzass merged successfully-------------")

# Merge orders_pizzas with pizza_types on 'pizza_type_id'
everything = pd.merge(orders_pizzas, pizza_types, on='pizza_type_id', how='inner')
print("--------everything merged successfuly------------------")


print(everything.head())

--------orders_orders_details merged successfully------
--------orders_pizzass merged successfully-------------
--------everything merged successfuly------------------
   order_details_id  order_id       pizza_id  quantity        date      time  \
0                 1         1     hawaiian_m         1  2015-01-01  11:38:36   
1                 2         2  classic_dlx_m         1  2015-01-01  11:57:40   
2                 3         2  five_cheese_l         1  2015-01-01  11:57:40   
3                 4         2    ital_supr_l         1  2015-01-01  11:57:40   
4                 5         2     mexicana_m         1  2015-01-01  11:57:40   

  pizza_type_id size  price                       name category  \
0      hawaiian    M  13.25         The Hawaiian Pizza  Classic   
1   classic_dlx    M  16.00   The Classic Deluxe Pizza  Classic   
2   five_cheese    L  18.50      The Five Cheese Pizza   Veggie   
3     ital_supr    L  20.75  The Italian Supreme Pizza  Supreme   
4      mexicana 

In [5]:
everything.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48620 entries, 0 to 48619
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   order_details_id  48620 non-null  int64  
 1   order_id          48620 non-null  int64  
 2   pizza_id          48620 non-null  object 
 3   quantity          48620 non-null  int64  
 4   date              48620 non-null  object 
 5   time              48620 non-null  object 
 6   pizza_type_id     48620 non-null  object 
 7   size              48620 non-null  object 
 8   price             48620 non-null  float64
 9   name              48620 non-null  object 
 10  category          48620 non-null  object 
 11  ingredients       48620 non-null  object 
dtypes: float64(1), int64(3), object(8)
memory usage: 4.5+ MB


In [6]:
everything["date"] = pd.to_datetime(everything["date"])
everything["time"] = pd.to_datetime(everything["time"], format='%H:%M:%S').dt.time

everything.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48620 entries, 0 to 48619
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   order_details_id  48620 non-null  int64         
 1   order_id          48620 non-null  int64         
 2   pizza_id          48620 non-null  object        
 3   quantity          48620 non-null  int64         
 4   date              48620 non-null  datetime64[ns]
 5   time              48620 non-null  object        
 6   pizza_type_id     48620 non-null  object        
 7   size              48620 non-null  object        
 8   price             48620 non-null  float64       
 9   name              48620 non-null  object        
 10  category          48620 non-null  object        
 11  ingredients       48620 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(3), object(7)
memory usage: 4.5+ MB


In [7]:
#-----------------MISSING VALUES---------------


print("--------------Missing values per table:-----------------")
print("--------------orders:-----------\n", orders.isnull().sum())
print("\n------------order_details:----\n", order_details.isnull().sum())
print("\n------------pizzas:-----------\n", pizzas.isnull().sum())
print("\n------------pizza_types:------\n", pizza_types.isnull().sum())

--------------Missing values per table:-----------------
--------------orders:-----------
 order_id    0
date        0
time        0
dtype: int64

------------order_details:----
 order_details_id    0
order_id            0
pizza_id            0
quantity            0
dtype: int64

------------pizzas:-----------
 pizza_id         0
pizza_type_id    0
size             0
price            0
dtype: int64

------------pizza_types:------
 pizza_type_id    0
name             0
category         0
ingredients      0
dtype: int64


In [8]:
print(everything.isnull().sum())
print(f"Everything shape: {everything.shape}")

order_details_id    0
order_id            0
pizza_id            0
quantity            0
date                0
time                0
pizza_type_id       0
size                0
price               0
name                0
category            0
ingredients         0
dtype: int64
Everything shape: (48620, 12)


In [9]:
#-----------TOTAL REVENUE----------

everything["revenue"] = everything["quantity"]*everything["price"] #revenue is the product of quantity & price
print("---------Revenue column created-------------")

total_revenue = everything["revenue"].sum()
print(f"Total revenue is  ${total_revenue:}")

---------Revenue column created-------------
Total revenue is  $817860.05


In [10]:
#---------TOTAL QUANTITY SOLD---------

total_quantity = everything["quantity"].sum()

print(f"The total Quantity sold for the year is ${total_quantity}")

The total Quantity sold for the year is $49574


In [11]:
#---------TOTAL ORDERS----------------

total_orders = everything["order_id"].nunique() #count the order items once to avoid duplicates

print(f"The Total Orders: {total_orders}")

The Total Orders: 21350


In [12]:
#--------AVERAGE PRICE OF PIZZAS------

avg_pizza_price = everything["price"].mean()
print(f"The average price of Pizzas is: ${avg_pizza_price: .2f}")

The average price of Pizzas is: $ 16.49


In [13]:
print(everything.head())

   order_details_id  order_id       pizza_id  quantity       date      time  \
0                 1         1     hawaiian_m         1 2015-01-01  11:38:36   
1                 2         2  classic_dlx_m         1 2015-01-01  11:57:40   
2                 3         2  five_cheese_l         1 2015-01-01  11:57:40   
3                 4         2    ital_supr_l         1 2015-01-01  11:57:40   
4                 5         2     mexicana_m         1 2015-01-01  11:57:40   

  pizza_type_id size  price                       name category  \
0      hawaiian    M  13.25         The Hawaiian Pizza  Classic   
1   classic_dlx    M  16.00   The Classic Deluxe Pizza  Classic   
2   five_cheese    L  18.50      The Five Cheese Pizza   Veggie   
3     ital_supr    L  20.75  The Italian Supreme Pizza  Supreme   
4      mexicana    M  16.00         The Mexicana Pizza   Veggie   

                                         ingredients  revenue  
0           Sliced Ham, Pineapple, Mozzarella Cheese    13

In [14]:
#-------PEAK HOURS OF SALE--------

everything["datetime"] = pd.to_datetime(everything["date"].astype(str) + " " + everything["time"].astype(str))
everything["hour"] = everything["datetime"].dt.hour

peak_hours = everything.groupby("hour")["revenue"].sum().reset_index().sort_values("revenue", ascending=False)

print(f"The peak hours of Sale are: {peak_hours}")

The peak hours of Sale are:     hour    revenue
3     12  111877.90
4     13  106065.70
9     18   89296.85
8     17   86237.45
10    19   72628.90
7     16   70055.40
5     14   59201.40
11    20   58215.40
6     15   52992.30
2     11   44935.80
12    21   42029.80
13    22   22815.15
14    23    1121.35
1     10     303.65
0      9      83.00


In [15]:
#-------TOTAL SALE ON EACH DAY OF THE WEEK--------

total_sale_per_day = everything.groupby(everything["datetime"].dt.date)["revenue"].sum().reset_index()
print(total_sale_per_day.head())


     datetime  revenue
0  2015-01-01  2713.85
1  2015-01-02  2731.90
2  2015-01-03  2662.40
3  2015-01-04  1755.45
4  2015-01-05  2065.95


In [16]:
#-------TOP 5 SELLING PIZZAS----------------------

top_pizzas = everything.groupby("name")["quantity"].sum().reset_index().sort_values("quantity", ascending=False)
print(f"The top 5 selling Pizzas are: \n {top_pizzas.head(5)}")

The top 5 selling Pizzas are: 
                           name  quantity
7     The Classic Deluxe Pizza      2453
0   The Barbecue Chicken Pizza      2432
12          The Hawaiian Pizza      2422
20         The Pepperoni Pizza      2418
30      The Thai Chicken Pizza      2371


In [ ]:
#-------SALES MADE IN EACH MONTH------------------

everything["month"] = everything["datetime"].dt.month # extracting the month number from datetime
everything["month_name"] = everything["datetime"].dt.strftime('%B') # extract month name 

sales_per_month = everything.groupby(everything["month_name"])["revenue"].sum().reset_index().sort_values("month_name", ascending=False)
print(f"The total Sale made in each month of the year is:\n {sales_per_month}")

In [ ]:
#-------PIZZA TYPES NOT DOING WELL ON THE MENU----

pizza_perform = everything.groupby("name")["quantity"].sum().reset_index().sort_values("quantity", ascending=True)
print(f"The least performing Pizza (Top 5) by sales are:\n {pizza_perform.head()}")

## Conclusion
- The least performing Pizza by quantity sold is "The Brie Carre Pizza". It is the only Pizza being sold on the menu with less than 500 sales